In [104]:
from keras.models import load_model
import os
import mediapipe as mp
import pandas as pd
import cv2
import numpy as np
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [116]:
data_path=r"C:\Users\Rohit Varre\Desktop\MliS\Python\SYN data"
categories=os.listdir(data_path)
labels = [i for i in range(len(categories))]
code=dict(zip(labels,categories))     # Dictionary to extract categories from labels
cap = cv2.VideoCapture(0)             # Opencv command to access the webcamera
flag_start_capturing = False

In [117]:
print(code)

{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h', 8: 'i', 9: 'j', 10: 'k', 11: 'l', 12: 'm', 13: 'n', 14: 'o', 15: 'p', 16: 'q', 17: 'r', 18: 's', 19: 't', 20: 'u', 21: 'v', 22: 'w'}


In [118]:
# Load the trained model
model = load_model('mediapipe.h5')
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 120)               7320      
_________________________________________________________________
dropout_13 (Dropout)         (None, 120)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 60)                7260      
_________________________________________________________________
dense_54 (Dense)             (None, 30)                1830      
_________________________________________________________________
dense_55 (Dense)             (None, 23)                713       
Total params: 17,123
Trainable params: 17,123
Non-trainable params: 0
_________________________________________________________________


In [119]:
# Mediapipe methods to extract and display hand cooridnates
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.4,
    max_num_hands=1,
    min_tracking_confidence=0.4
)

In [120]:
# Loops over the frames during the video
while True:
    ret,frame = cap.read()
    image = cv2.flip(frame,1)
    x = []
    y = []
    z = []
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = True
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for i in hand_landmarks.landmark:
                x.append(i.x)
                y.append(i.y)
                z.append(i.z)

    keypress = cv2.waitKey(5)
    if keypress == 32:                                # Click spacebar to start or stop predictions
        if flag_start_capturing == False:
            flag_start_capturing = True
        else:
            flag_start_capturing = False
    if flag_start_capturing == True and results.multi_hand_landmarks:
        df = pd.DataFrame({'x':x,'y':y,'z':z})
        data = np.array(df)
        data = data-data[0]                           # Same data scaling done during training
        data = data[1:]
        data = np.reshape(data,(-1,60))
        out = model.predict(data)
        output=np.argmax(out,axis=1)[0]
        cv2.putText(image, code[output], (30,400), cv2.FONT_HERSHEY_TRIPLEX, 2.5, (127,127,255))
        
    cv2.imshow('MediaPipe Hands', image)
    
    if keypress == 27:                                # Click escape to end the program
        break
cv2.destroyAllWindows()
cap.release()